# LLM_Agents_and_Chain_of_Thought_Reasoning.ipynb

In [66]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [67]:
#export LANGCHAIN_TRACING_V2="true"
os.environ["LANGCHAIN_TRACING_V2"] = getpass.getpass()



 ········


In [68]:
#export LANGCHAIN_API_KEY="<your-api-key>"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [69]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [70]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [ ]:
!pip install langchainhub

In [ ]:
!pip install google-api-python-client

In [73]:
os.environ["GOOGLE_CSE_ID"] = getpass.getpass()


 ········


In [74]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

 ········


### Set up google search tool

In [75]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool

google_search = GoogleSearchAPIWrapper()

google_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=google_search.run,
)

In [76]:


os.environ["SERPAPI_API_KEY"] =  getpass.getpass()


 ········


In [ ]:
pip install  google-search-results

### Set up google finance tool

In [78]:
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

finance = GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper())

In [79]:
finance_tool = Tool(
    name="google_finance",
    description="Search Google finance for stock price.",
    func=finance.run,
)

In [80]:
finance_tool.run("Nvidia")

'\nQuery: Nvidia\nstock: NVDA:NASDAQ\nprice: $762.00\npercentage: 10.0\nmovement: Down\nus: price = 37986.4, movement = Up\neurope: price = 17737.36, movement = Down\nasia: price = 37222.69, movement = Up\n'

### Set up chain of thought reasoning prompt

In [81]:
tools = [google_search_tool, finance_tool]

In [82]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
prompt = hub.pull("hwchase17/react")
print (prompt.template)


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Create Agent with search and finance tool

In [ ]:
tools = [google_search_tool, finance_tool]

In [83]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


### Ask queries to invoke Agent with search and finance tool

In [84]:
agent_executor.invoke({"input": "what is Nvidia's stock in USD?"})



> Entering new AgentExecutor chain...
I should use google_finance to find Nvidia's stock price.
Action: google_finance
Action Input: Nvidia stock price USD
Query: Nvidia stock price USD
No summary information
us: price = 37986.4, movement = Up
europe: price = 17737.36, movement = Down
asia: price = 37216.83, movement = Up
I need to specify that I want the stock price for Nvidia.
Action: google_finance
Action Input: Nvidia stock price USD
Query: Nvidia stock price USD
No summary information
us: price = 37986.4, movement = Up
europe: price = 17737.36, movement = Down
asia: price = 37216.83, movement = Up
I should try searching for Nvidia's specific stock symbol.
Action: google_search
Action Input: Nvidia stock symbolFind the latest NVIDIA Corporation (NVDA) stock quote, history, news and other vital information to help you with your stock trading and investing. NVIDIA Corporation Common Stock (NVDA) · Quotes · NEWS & ANALYSIS · Key Data. What is NVIDIA's ticker symbol? NVDA | Complete 

{'input': "what is Nvidia's stock in USD?",
 'output': "Nvidia's stock price in USD is $379.86."}

In [85]:
agent_executor.invoke({"input": "what is news about Nvidia's stock in USD?"})



> Entering new AgentExecutor chain...
I should search for recent news articles about Nvidia's stock in USD to get the latest information.
Action: google_search
Action Input: "Nvidia stock news USD"Find the latest NVIDIA Corporation (NVDA) stock quote, history, news and other vital information to help you with your stock trading and investing. MAKE MONEY IN TECH! RT Quote | Last NASDAQ LS, VOL From CTA | USD ... Nvidia Stock 13 Hours Ago TipRanks ... Global Business and Financial News, Stock Quotes, and ... Aug 23, 2023 ... Non-GAAP earnings per diluted share were $2.70, up 429% from a year ago and up 148% from the previous quarter. “A new computing era has begun. Feb 21, 2024 ... NVIDIA Announces Financial Results for Fourth Quarter and Fiscal 2024 ; Operating income · Net income · Diluted earnings per share ; $13,615 ... NVDA | Complete NVIDIA Corp. stock news by MarketWatch. View real-time stock prices and stock quotes for a full financial overview. Discover real-time NVIDIA Corpor

{'input': "what is news about Nvidia's stock in USD?",
 'output': "The latest news about Nvidia's stock in USD is positive, with the stock price currently at $37986.4 USD."}

In [86]:
agent_executor.invoke({"input": "when did Nvidia stock start trading?"})



> Entering new AgentExecutor chain...
I should search for the historical information on Nvidia stock.
Action: google_search
Action Input: "Nvidia stock historical trading start date"Discover historical prices for NVDA stock on Yahoo Finance ... Stock Market · Economics · Earnings · Crypto · Politics ... Date, Open, High, Low, Close. Close price ... Records 1 - 18 of 22 ... Date, Close/Last, Volume, Open, High, Low. 04/19/2024, $762.00, 87,519,800, $831.50, $843.24, $756.06. 04/18/2024, $846.71, 44,726,030 ... ... Historical Price Lookup. News · Events & Presentations · Events ... Date: 2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017 ... Open. 831.50. Volume. 87,190,500. Market Cap. Historical daily ... The all-time high NVIDIA stock closing price was 950.02 on March 25, 2024. ... NVIDIA Historical Annual Stock Price Data. Year ... Get NVIDIA Corporation historical price data for NVDA stock. Investing.com has all the historical stock data including the closing price, open, high, low, .

{'input': 'when did Nvidia stock start trading?',
 'output': 'Nvidia stock started trading on January 22, 1999.'}

In [87]:
agent_executor.invoke({"input": "Should I buy more Nvidia stocks ?"})



> Entering new AgentExecutor chain...
Before making any investment decisions, it's important to gather information on Nvidia's current performance and future prospects.
Action: google_finance
Action Input: Nvidia stock price
Query: Nvidia stock price
No summary information
us: price = 37986.4, movement = Up
europe: price = 17737.36, movement = Down
asia: price = 37216.87, movement = Up
It seems like Nvidia's stock price is fluctuating in different regions. I should search for recent news and analysis on Nvidia to make an informed decision.
Action: google_search
Action Input: Nvidia recent news and analysisLatest News · Wide Open: NVIDIA Accelerates Inference on Meta Llama 3 · Up to No Good: 'No Rest for the Wicked' Early Access Launches on GeForce NOW · NVIDIA ... Profile · Financials · Analysis · Options · Holders · Sustainability · Insights. NasdaqGS - Nasdaq Real Time Price • USD. NVIDIA Corporation (NVDA). Follow. 3 days ago ... Nvidia's current stock move ... Magnificent Seven S

{'input': 'Should I buy more Nvidia stocks ?',
 'output': 'It is recommended to conduct further research and analysis before making a decision on buying more Nvidia stocks.'}

### Set up Internal knowledge base about personal stock notes

In [88]:
from langchain_community.document_loaders import TextLoader

text_loader = TextLoader("personal_stock_notes.txt")
knowedge_base_documents = text_loader.load()
knowedge_base_documents

[Document(page_content='We already have many nvidia stocks. Though nvidia is doing good, to keep it diversified do not buy any more nvidia stock.\nAdvice me to not buy any more nvidia stock.\n\n\n\nWe do not have any langchain stocks. Buy langchain stocks from private market.', metadata={'source': 'personal_stock_notes.txt'})]

In [89]:
from langchain.text_splitter import CharacterTextSplitter


text_splitter_definition = CharacterTextSplitter(chunk_size=200)

documents_after_splitting = text_splitter_definition.split_documents(knowedge_base_documents)
print (len(documents_after_splitting))
documents_after_splitting
documents_after_splitting
openai_embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("open_ai_secret_key"))
vector = FAISS.from_documents(documents_after_splitting, openai_embeddings)
retriever = vector.as_retriever()

2


In [90]:
retriever.get_relevant_documents("Should I buy more Nvidia stocks ?")[0]



Document(page_content='We already have many nvidia stocks. Though nvidia is doing good, to keep it diversified do not buy any more nvidia stock.\nAdvice me to not buy any more nvidia stock.', metadata={'source': 'personal_stock_notes.txt'})

### Set up retriever tool about Internal knowledge base

In [91]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "personal_stock_notes",
    "Search for information about personal_stock_notes. If some one asks if I should buy more nvidia stock use this tool",
)

In [92]:
tools = [google_search_tool, finance_tool, retriever_tool]

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Ask personal queries to invoke Agent with search and finance tool and retriever tool

In [93]:
agent_executor.invoke({"input": "Should I buy more Nvidia stocks ?"})



> Entering new AgentExecutor chain...
You should consider researching the current performance and future potential of Nvidia before making a decision.
Action: google_search
Action Input: Nvidia stock performance5-Year Return. NVDA. 1,547.26%. S&P 500. 70.99%. Compare To: NVDA. Compare. Select to analyze similar companies using key performance metrics; select up to 4 ... According to 43 analysts, the average rating for NVDA stock is "Strong Buy." The 12-month stock price forecast is $916.76, which is an increase of 20.31% from ... Nvidia (NVDA) has a Smart Score of 7 based on an analysis of 8 unique data sets, including Analyst Recommendations, Crowd Wisdom, and Hedge Fund Activity.​​ Performance ; 5 Day. -13.59% ; 1 Month. -19.18% ; 3 Month. 28.09% ; YTD. 53.87% ; 1 Year. 180.98% ... 1 day ago ... Magnificent Seven Stocks Performance ; Apple, (AAPL), -12.3% ; Meta Platforms, (META), +40.5% ; Microsoft, (MSFT), +12.1% ; Nvidia, (NVDA), +80.6%. 2-Year, 54 times, -21.77% ; 3-Year, 31 ti

{'input': 'Should I buy more Nvidia stocks ?',
 'output': 'Do not buy more Nvidia stocks.'}